In [92]:
import pandas as pd

In [93]:
!pip3 install chardet

In [94]:
pwd

'/Users/jm/Desktop/ironhack/curso/shark-project'

In [95]:
import chardet

with open('./Input/GSAF5.csv', 'rb') as f:
    result = chardet.detect(f.read())  # or readline if the file is large


df = pd.read_csv('./Input/GSAF5.csv', encoding=result['encoding'])
print(df.shape)

(5992, 24)


In [96]:
#Busco los nulos existentes en las columnas
null_cols = df.isnull().sum()
null_cols

Case Number                  0
Date                         0
Year                         0
Type                         0
Country                     43
Area                       402
Location                   496
Activity                   527
Name                       200
Sex                        567
Age                       2681
Injury                      27
Fatal (Y/N)                 19
Time                      3213
Species                   2934
Investigator or Source      15
pdf                          0
href formula                 1
href                         3
Case Number.1                0
Case Number.2                0
original order               0
Unnamed: 22               5991
Unnamed: 23               5990
dtype: int64

In [97]:
#Elimino unnamed22 y unnamed 23 porque no continen datos
drop_cols = list(null_cols[null_cols > 5000].index)
df = df.drop(drop_cols, axis=1)
print(df.shape)

(5992, 22)


In [98]:
#Elimino CaseNumber.1 y CaseNumber.2 porque son un duplicado de la primera columna CaseNumber
df = df.drop(columns=['Case Number.1', 'Case Number.2'])
print(df.shape)

(5992, 20)


In [99]:
''' 
Sale False porque no son exactamente iguales ya que al ver cuantos nulos hay en cada uno en una salen más que en otra,
pero observando los datos las URL son iguales, por lo que borramos la que más nulos tiene.
'''
df['href formula'].equals(df['href'])

df = df.drop(columns=['href'])
print(df.shape)

(5992, 19)


In [100]:
#La columna pdf se refiere al pdf que surje de la URL por lo que no nos da información extra.
df = df.drop(columns=['pdf'])
print(df.shape)

(5992, 18)


In [101]:
df["Fatal (Y/N)"].value_counts().sort_index()

 N            8
#VALUE!       1
F             1
N          4315
N             1
UNKNOWN      94
Y          1552
n             1
Name: Fatal (Y/N), dtype: int64

In [128]:
#Unifico valores en la columna Fatal para saber cuantos han sido mortales y cuales no
df = df.rename(columns={"Fatal (Y/N)":"Fatal"})
df.Fatal = df.Fatal.replace({"N": "N", "n": "N", "#VALUE!": "N", "F":"Y", "UNKNOWN":"N", " N": "N", "N ": "N", "Y": "Y"})


In [129]:
df["Fatal"].value_counts().sort_index()

N    4111
Y    1201
Name: Fatal, dtype: int64

In [104]:
df

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal,Time,Species,Investigator or Source,href formula,original order
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,16,Minor injury to thigh,N,13h00,NaN,"Orlando Sentinel, 9/19/2016",http://sharkattackfile.net/spreadsheets/pdf_di...,5993
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,36,Lacerations to hands,N,11h00,NaN,"Orlando Sentinel, 9/19/2016",http://sharkattackfile.net/spreadsheets/pdf_di...,5992
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,43,Lacerations to lower leg,N,10h43,NaN,"Orlando Sentinel, 9/19/2016",http://sharkattackfile.net/spreadsheets/pdf_di...,5991
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,NaN,Struck by fin on chest & leg,N,NaN,NaN,"The Age, 9/18/2016",http://sharkattackfile.net/spreadsheets/pdf_di...,5990
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,NaN,No injury: Knocked off board by shark,N,NaN,2 m shark,"The Age, 9/16/2016",http://sharkattackfile.net/spreadsheets/pdf_di...,5989
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5987,ND.0005,Before 1903,0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, p. 234",http://sharkattackfile.net/spreadsheets/pdf_di...,6
5988,ND.0004,Before 1903,0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, pp. 233-234",http://sharkattackfile.net/spreadsheets/pdf_di...,5
5989,ND.0003,1900-1905,0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,NaN,FATAL,Y,NaN,NaN,"F. Schwartz, p.23; C. Creswell, GSAF",http://sharkattackfile.net/spreadsheets/pdf_di...,4
5990,ND.0002,1883-1889,0,Unprovoked,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,Jules Patterson,M,NaN,FATAL,Y,NaN,NaN,"The Sun, 10/20/1938",http://sharkattackfile.net/spreadsheets/pdf_di...,3


In [105]:
df["Year"].value_counts().sort_index()


0       124
5         1
77        1
500       1
1543      1
       ... 
2012    117
2013    122
2014    125
2015    139
2016    103
Name: Year, Length: 232, dtype: int64

In [106]:
#Elimino las filas con un año inferior a 1850 porque la gente de esa época bebía mucho y no hacia bien los registros :)
df = df.drop(df[df['Year']<1850].index)
df

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal,Time,Species,Investigator or Source,href formula,original order
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,16,Minor injury to thigh,N,13h00,NaN,"Orlando Sentinel, 9/19/2016",http://sharkattackfile.net/spreadsheets/pdf_di...,5993
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,36,Lacerations to hands,N,11h00,NaN,"Orlando Sentinel, 9/19/2016",http://sharkattackfile.net/spreadsheets/pdf_di...,5992
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,43,Lacerations to lower leg,N,10h43,NaN,"Orlando Sentinel, 9/19/2016",http://sharkattackfile.net/spreadsheets/pdf_di...,5991
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,NaN,Struck by fin on chest & leg,N,NaN,NaN,"The Age, 9/18/2016",http://sharkattackfile.net/spreadsheets/pdf_di...,5990
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,NaN,No injury: Knocked off board by shark,N,NaN,2 m shark,"The Age, 9/16/2016",http://sharkattackfile.net/spreadsheets/pdf_di...,5989
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5751,1852.00.00,1852,1852,Unprovoked,USA,South Carolina,"Mount Pleasant, Charleston County","Vessel capsized, wading ashore carrying an oar",Charles Chambers,M,NaN,"FATAL, body was not recovered",Y,NaN,NaN,"W. H. Gregg, p. 21",http://sharkattackfile.net/spreadsheets/pdf_di...,242
5752,1851.06.19.R,Reported 19-Jun-1851,1851,Unprovoked,MEXICO,Oaxaca,NaN,Bathing,John Gray & another member of the Tehuantepec ...,M,NaN,FATAL,Y,NaN,NaN,"Burlington Hawk-Eye, 6/19/1851",http://sharkattackfile.net/spreadsheets/pdf_di...,241
5753,1851.03.08.R,Mar-1851,1851,Unprovoked,USA,Hawaii,Honolulu Harbor,Swimming,James Kinney,M,NaN,FATAL,Y,NaN,NaN,"The Friend (Honolulu), 3/8/1851",http://sharkattackfile.net/spreadsheets/pdf_di...,240
5754,1851.00.00,1851,1851,Unprovoked,USA,California,"San Francisco Bay (or San Leandro Bay), near c...",Hard hat diving,William Cortigan,M,NaN,3 toes severed,N,NaN,18' shark,"The Perry Chief, 10/16/1875",http://sharkattackfile.net/spreadsheets/pdf_di...,239


In [107]:
nulas_nuevas = df.isnull().sum()
nulas_nuevas

Case Number                  0
Date                         0
Year                         0
Type                         0
Country                     34
Area                       349
Location                   435
Activity                   476
Name                       186
Sex                        543
Age                       2468
Injury                      20
Fatal                       15
Time                      2993
Species                   2743
Investigator or Source      13
href formula                 1
original order               0
dtype: int64

In [108]:
df["Year"].value_counts().sort_index()

1850      1
1851      3
1852     10
1853      7
1854      1
       ... 
2012    117
2013    122
2014    125
2015    139
2016    103
Name: Year, Length: 166, dtype: int64

In [131]:
#Ataques últimos 50 años de 1800 son entre 1-20 al año por lo que no es muy representativo
x = df[(df["Year"] <1900)]
x["Year"].value_counts().sort_index()
df = df.drop(df[df['Year']<1900].index)
y = df[(df["Year"] <1950)]
y["Year"].value_counts().sort_index()
#Ataques primeros 50 años de 1900 son como mucho 40 al año por lo que no es muy representativo
df = df.drop(df[df['Year']<1950].index)
z = df[(df["Year"] <2000)]
z["Year"].value_counts().sort_index()

1950    43
1951    31
1952    29
1953    36
1954    42
1955    43
1956    51
1957    41
1958    54
1959    93
1960    93
1961    78
1962    86
1963    61
1964    66
1965    51
1966    58
1967    48
1968    46
1969    30
1970    42
1971    28
1972    35
1973    27
1974    38
1975    49
1976    39
1977    26
1978    25
1979    25
1980    35
1981    49
1982    40
1983    50
1984    41
1985    37
1986    39
1987    35
1988    55
1989    53
1990    38
1991    38
1992    56
1993    56
1994    56
1995    76
1996    61
1997    57
1998    65
1999    65
Name: Year, dtype: int64

In [110]:
#Renombro los paises mal escritos
df["Country"].value_counts()
df.Country = df.Country.replace({" PHILIPPINES": "PHILIPPINES", " TONGA": "TONGA"})

In [111]:
#Agrupo boating y boat
df.Type = df.Type.replace({"Boating": "Boat"})

In [112]:
#Quito el espacio del nombre de la columna y unifico valores
df = df.rename(columns={"Sex ":"Sex"})
df["Sex"].value_counts()
df.Sex = df.Sex.replace({"M ": "M"})


In [113]:
df["Sex"].value_counts()

M      4642
F       567
.         1
lli       1
N         1
Name: Sex, dtype: int64

In [118]:
#Elimino los datos que no me interesan ya que quiero saber por ataques a hombres o a mujeres
df = df.drop(df[(df['Sex']==".")].index)
df = df.drop(df[(df['Sex']=="lli")].index)
df = df.drop(df[(df['Sex']=="N")].index)


In [119]:
df["Sex"].value_counts()

M    4642
F     567
Name: Sex, dtype: int64